In [1]:
#import libraries needed for this proces
import requests
from datetime import datetime
import numpy
import time
import json
import pandas as pd

In [ ]:
#function to create an API request based on URL and  parameters
def get_request(url, parameters=None):
    print("Your request is being processed.")
    response = requests.get(url=url, params=parameters)
    if response:
        return response.json()
        
    else:
        print("Request unsuccessful, retrying in 5 seconds.")
        time.sleep(5)
        return get_request(url, parameters)
    time.sleep(1)



#using the function to get all data id's which include appid and app name 
url = "https://api.steampowered.com/ISteamApps/GetAppList/v2/"
parameters = {"request": all}
data_ids = get_request(url, parameters=parameters)
print("Your request has been processed successfully, call upon 'data_ids' to preview the full list of IDs and names of all currently available products on Steam as of ", datetime.now().strftime("%d/%m/%Y %H:%M:%S"), ".")

data_ids

In [68]:
#Code to collect all app id's in one library
app_ids = []
print("Creating a list of all IDs currently available on Steam")
for item in data_ids['applist']['apps']:
    app_ids.append(item['appid'])

print("List created sucessfully, the total number of IDs available on Steam is:", len(app_ids),".")
print("Call upon 'app_ids' to preview the full list of extracted IDs.")



Creating a list of all IDs currently available on Steam
List created sucessfully, the total number of IDs available on Steam is: 138116 .
Call upon 'app_ids' to preview the full list of extracted IDs.


In [39]:
# example code that collects the details of 1 app by appid  
url = "http://store.steampowered.com/api/appdetails/"
parameters = {"appids": "10"}
app_data = get_request(url, parameters=parameters)
app_data["10"]['data']


Your request is being processed.


{'type': 'game',
 'name': 'Counter-Strike',
 'steam_appid': 10,
 'required_age': 0,
 'is_free': False,
 'detailed_description': "Play the world's number 1 online action game. Engage in an incredibly realistic brand of terrorist warfare in this wildly popular team-based game. Ally with teammates to complete strategic missions. Take out enemy sites. Rescue hostages. Your role affects your team's success. Your team's success affects your role.",
 'about_the_game': "Play the world's number 1 online action game. Engage in an incredibly realistic brand of terrorist warfare in this wildly popular team-based game. Ally with teammates to complete strategic missions. Take out enemy sites. Rescue hostages. Your role affects your team's success. Your team's success affects your role.",
 'short_description': "Play the world's number 1 online action game. Engage in an incredibly realistic brand of terrorist warfare in this wildly popular team-based game. Ally with teammates to complete strategic mis

In [ ]:
#it finaly loops
#however it gives some succes and some false values. so we need to build in a check if succes = false -> then next. if succes = True -> write the data to a Csv 
for item in app_ids:
    url_appdetails= "http://store.steampowered.com/api/appdetails/"
    parameters = {"appids": {item}}
    app_details = get_request(url_appdetails, parameters=parameters)
    print(app_details)

In [ ]:
## collecting the data with a for loop that checks if the response was a succes = True. and than takes the data attribute 
## with addition of try and excepts from michael 

details = {} 

counter = 1   
for i in app_ids: 
    url = "http://store.steampowered.com/api/appdetails/"
    parameters = {"appids": {i}}
    app_data = get_request(url, parameters=parameters)
    id = str(i)
    print("Processing data pertaining to Steam ID:"+id)
    response = app_data[id]
    succes = response["success"]
    if (succes): 
        data_json = response["data"]
        try:
            app_id = data_json["steam_appid"]
            name = data_json["name"]
            prod_type = data_json["type"]
            freemium = data_json["is_free"]
            price = data_json["price_overview"]["final_formatted"]
            discount = data_json["price_overview"]["discount_percent"]
            developer = data_json["developers"]
            publisher = data_json["publishers"]
            desc = data_json["detailed_description"]
            short_desc = data_json["short_description"]
            try:
                release_date = data_json["release_date"]["date"]
            except:
                release_date = "TBA"
            try:
                win = data_json["platforms"]["windows"]
            except:
                win = ""
            try:
                mac = data_json["platforms"]["mac"]
            except:
                mac = ""
            try:
                lin = data_json["platforms"]["linux"]
            except:
                lin = ""
            try:
                achievements = data_json["achievements"]["total"]
            except:
                achievements = "0"
            try:
                reviews = data_json["recommendations"]
            except:
                reviews = ""
            try:
                metacritic = data_json["metacritic"]
            except:
                metacritic = "not available"
            try:
                base_app = data_json["fullgame"]["appid"]
            except:
                base_app = "none"
            scrape_datetime = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
            details[app_id]=[name],[release_date],[freemium],[prod_type],[price],[discount],[developer],[publisher],[desc],[short_desc],[achievements],[win],[mac],[lin],[reviews],[metacritic],[base_app],[scrape_datetime]
        except:
            pass
        counter = counter + 1 
    if (counter > 10000):
        break           


In [ ]:
## collecting the data with a for loop that checks if the response was a succes = True. and than takes the data attribute 
details = {} 
dataframe = pd.DataFrame()

amount_app_ids = 100000
counter = 1   
for i in app_ids: 
    url = "http://store.steampowered.com/api/appdetails/"
    parameters = {"appids": {i}}
    app_data = get_request(url, parameters=parameters)
    id = str(i)
    print("Processing data pertaining to Steam ID:"+id)
    response = app_data[id]
    succes = response["success"]
    if (succes): 
        data_json = response["data"]
        result = pd.json_normalize(data_json)
        #categories = pd.json_normalize(data_json,record_path= ['categories'])
        dataframe = pd.concat([dataframe, result],ignore_index= True)
        counter = counter + 1 
    if (counter > amount_app_ids):
        break           


In [74]:
dataframe
#dataframe.to_excel("D:\\1. Studie\\Master files\\test_df.xlsx",)

#pd.json_normalize(data_json)

,type,name,steam_appid,required_age,is_free,detailed_description,about_the_game,short_description,supported_languages,header_image,...,mac_requirements.recommended,linux_requirements.minimum,linux_requirements.recommended,controller_support,dlc,legal_notice,demos,ext_user_account_notice,reviews,pc_requirements
0,game,Tales of the Black Death,1590000,0,False,"<img src=""https://cdn.akamai.steamstatic.com/s...","<img src=""https://cdn.akamai.steamstatic.com/s...",An interactive story that pins your morality a...,"English, French",https://cdn.akamai.steamstatic.com/steam/apps/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,game,LaserShotz,1590020,0,True,LaserShotz is an online PvP competitive arena ...,LaserShotz is an online PvP competitive arena ...,LaserShotz is an online PvP competitive arena ...,English<strong>*</strong><br><strong>*</strong...,https://cdn.akamai.steamstatic.com/steam/apps/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,game,Endless Furry Ping-Pong,1590030,0,True,Welcome to Endless Furry Ping-Pong!<br>The 7th...,Welcome to Endless Furry Ping-Pong!<br>The 7th...,Welcome to Endless Furry Ping-Pong! The 7th in...,English,https://cdn.akamai.steamstatic.com/steam/apps/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,demo,Sigma Draconis Demo,1590060,0,True,"This is the demo for Sigma Draconis, a sci-fi ...","This is the demo for Sigma Draconis, a sci-fi ...","This is the demo for Sigma Draconis, a sci-fi ...","English, German",https://cdn.akamai.steamstatic.com/steam/apps/...,...,"<strong>Recommended:</strong><br><ul class=""bb...","<strong>Minimum:</strong><br><ul class=""bb_ul""...","<strong>Recommended:</strong><br><ul class=""bb...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,game,Project Hunter,1590070,0,True,Stealth and tactic game that give you the cont...,Stealth and tactic game that give you the cont...,Stealth and tactic game that give you the cont...,English<strong>*</strong><br><strong>*</strong...,https://cdn.akamai.steamstatic.com/steam/apps/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,game,Gaslamp Cases: The deadly Machine,1588510,0,False,Morgan Johnson is one of Londons finest police...,Morgan Johnson is one of Londons finest police...,Morgan Johnson is one of Londons finest police...,"English<strong>*</strong>, French, German<br><...",https://cdn.akamai.steamstatic.com/steam/apps/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,game,SPECIAL FORCE DOUBLE TAKE,1588540,0,False,Survival justifies any means! As one of the sp...,Survival justifies any means! As one of the sp...,[SPECIAL FORCE DOUBLE TAKE] is an innovative 3...,English,https://cdn.akamai.steamstatic.com/steam/apps/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,game,The Swordsmen X: Survival,1588560,0,False,It’s a martial art open world adventure game t...,It’s a martial art open world adventure game t...,It’s a martial art open world adventure game t...,"Simplified Chinese, English",https://cdn.akamai.steamstatic.com/steam/apps/...,...,"<strong>Recommended:</strong><br><ul class=""bb...","<strong>Minimum:</strong><br><ul class=""bb_ul""...","<strong>Recommended:</strong><br><ul class=""bb...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,game,Lacuna: Prologue,1588570,0,True,"<h1>WISHLIST THE MAIN GAME</h1><p><a href=""htt...","<img src=""https://cdn.akamai.steamstatic.com/s...",Lacuna: Prologue tasks you – CDI agent Neil Co...,"English, German, Japanese, Korean, Simplified ...",https://cdn.akamai.steamstatic.com/steam/apps/...,...,"<strong>Recommended:</strong><br><ul class=""bb...","<strong>Minimum:</strong><br><ul class=""bb_ul""...","<strong>Recommended:</strong><br><ul class=""bb...",full,NaN,Copyright Assemble Entertainment GmbH 2021 - a...,NaN,NaN,NaN,NaN


In [ ]:
### example to create a dictionary that checks if the id is already in the dictionary and then if not in dictionary get data 

"""
details = {}

for i in app_ids:
  id = str(i)
  if id not in details:
     // get data

     if(success):
       // put data in details
       details[id] = data

"""     

In [ ]:
#not relevant yet 
#trying to make a function out of it


"""
this is a comment 

def Get_tha_data(attributes, limit):
 
counter = 1   
for i in app_ids: 
    url = "http://store.steampowered.com/api/appdetails/"
    parameters = {"appids": {i}}
    app_data = get_request(url, parameters=parameters)       
    id = str(i)
    response = app_data[id]
    succes = response["success"]
    if (succes): 
        data_json = response["data"]
        print(data_json)
        counter = counter + 1 
    if (counter > limit):
        break           
"""